Provide the **most optimized, production-quality, and interview-ready Rust solution** for the following DSA problem:

### Problem:

3362. Zero Array Transformation III

Medium
Topics
Companies
Hint
You are given an integer array nums of length n and a 2D array queries where queries[i] = [li, ri].

Each queries[i] represents the following action on nums:

Decrement the value at each index in the range [li, ri] in nums by at most 1.
The amount by which the value is decremented can be chosen independently for each index.
A Zero Array is an array with all its elements equal to 0.

Return the maximum number of elements that can be removed from queries, such that nums can still be converted to a zero array using the remaining queries. If it is not possible to convert nums to a zero array, return -1.

 

Example 1:

Input: nums = [2,0,2], queries = [[0,2],[0,2],[1,1]]

Output: 1

Explanation:

After removing queries[2], nums can still be converted to a zero array.

Using queries[0], decrement nums[0] and nums[2] by 1 and nums[1] by 0.
Using queries[1], decrement nums[0] and nums[2] by 1 and nums[1] by 0.
Example 2:

Input: nums = [1,1,1,1], queries = [[1,3],[0,2],[1,3],[1,2]]

Output: 2

Explanation:

We can remove queries[2] and queries[3].

Example 3:

Input: nums = [1,2,3,4], queries = [[0,3]]

Output: -1

Explanation:

nums cannot be converted to a zero array even after using all the queries.

 

Constraints:

1 <= nums.length <= 105
0 <= nums[i] <= 105
1 <= queries.length <= 105
queries[i].length == 2
0 <= li <= ri < nums.length

### Typing

```rust

```

---

### Requirements:

🔹 **Optimal Time Complexity**

- State the algorithm's time complexity and justify why it is optimal given the problem constraints.

🔹 **Optimal Space Complexity**

- Minimize extra space usage and explain the space complexity.

🔹 **Rust Code Quality**

- Use **clean, idiomatic, modular Rust code** suitable for **FAANG/MAANG interviews**.
- Follow **ownership, borrowing, and iterator best practices**.
- Include **realistic interview-level function signatures** (`fn solve(...) -> ...`).
- Avoid unnecessary `unwrap()`, `expect()`, or unsafe code unless clearly justified.
- Handle all **edge cases** thoughtfully within the code.
- Provide **unit tests** (`#[test]`) for common and edge cases.

🔹 **Algorithm Explanation**

- Describe the **core DSA concepts** used (e.g., Greedy, DP, Binary Search).
- Explain the **algorithm design, reasoning**, and why this is the most optimal approach.
- Discuss any **critical trade-offs** made.
- Identify common **edge cases** and explain how they are handled.

🔹 **Constraints & Scalability**

- Analyze the solution's behavior under **maximum input sizes**.
- Confirm it passes **stress tests** without significant performance degradation.

🔹 **DSA Tags**

- Suggest relevant DSA topics (e.g., Arrays, Trees, Graphs, Dynamic Programming, Sliding Window).

🔹 **Follow-up Enhancements (Optional)**

- How would the solution adapt to **streaming input**, **parallelization**, or **distributed systems**?
- What are potential **real-world applications** of this approach?


```py
```

In [ ]:
impl Solution {
    pub fn max_removal(nums: Vec<i32>, queries: Vec<Vec<i32>>) -> i32 {
        let n = nums.len();
        let q = queries.len();
        let mut cap = vec![0; n];
        for query in &queries {
            let l = query[0] as usize;
            let r = query[1] as usize;
            cap[l] += 1;
            if r + 1 < n {
                cap[r + 1] -= 1;
            }
        }
        for i in 1..n {
            cap[i] += cap[i - 1];
        }
        for i in 0..n {
            if cap[i] < nums[i] {
                return -1;
            }
        }
        let mut qs: Vec<(usize, usize)> = queries
            .into_iter()
            .map(|v| (v[0] as usize, v[1] as usize))
            .collect();
        qs.sort_by_key(|&(l, _)| l);
        let mut fenw = Fenw::new(n);
        let mut kept = 0;
        let mut j = 0;
        let mut heap = std::collections::BinaryHeap::new();
        for i in 0..n {
            while j < qs.len() && qs[j].0 == i {
                let (l, r) = qs[j];
                heap.push((r, l));
                j += 1;
            }
            let mut curr = fenw.query(i);
            while curr < nums[i] {
                if let Some((r, l)) = heap.pop() {
                    if r < i {
                        continue;
                    }
                    fenw.update(l, 1);
                    if r + 1 < n {
                        fenw.update(r + 1, -1);
                    }
                    kept += 1;
                    curr += 1;
                } else {
                    return -1;
                }
            }
        }
        (q - kept) as i32
    }
}

struct Fenw {
    n: usize,
    data: Vec<i32>,
}

impl Fenw {
    fn new(n: usize) -> Self {
        Fenw {
            n,
            data: vec![0; n + 1],
        }
    }
    fn update(&mut self, mut i: usize, delta: i32) {
        i += 1;
        while i <= self.n {
            self.data[i] += delta;
            i += i & i.wrapping_neg();
        }
    }
    fn query(&self, mut i: usize) -> i32 {
        let mut sum = 0;
        i += 1;
        while i > 0 {
            sum += self.data[i];
            i -= i & i.wrapping_neg();
        }
        sum
    }
}

```py
class Solution:
    def maxRemoval(self, nums: List[int], queries: List[List[int]]) -> int:
        n, q = len(nums), len(queries)
        starts = [[] for _ in range(n)]
        for l, r in queries:
            starts[l].append(r)

        avail = []   # max‐heap of ends (store negatives)
        active = []  # min‐heap of ends
        chosen = 0
        for i in range(n):
            for r in starts[i]:
                heapq.heappush(avail, -r)

            while active and active[0] < i:
                heapq.heappop(active)

            need = nums[i] - len(active)
            for _ in range(need):
                while avail and -avail[0] < i:
                    heapq.heappop(avail)
                if not avail:
                    return -1
                r = -heapq.heappop(avail)
                heapq.heappush(active, r)
                chosen += 1
        return q - chosen
```